# Convert JSON to TEXT INDEX for RAG on HATVP declarations

Now on only use an index that is the name and surname and job title of the person doing the declaration document.  

This is the "metadata" that we inserted into the text previously, now we only use this.

## install libs

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

declarations_ds = load_dataset('the-french-artist/hatvp_declarations_xml_plus_json', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

We remove the previous extracted text feature (all text contained in the JSON) as we will replace this feature with the text index

In [ ]:
declarations_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'declaration_json', 'extracted_text'],
    num_rows: 10944
})

In [ ]:
declarations_ds = declarations_ds.remove_columns(["extracted_text"])

In [ ]:
declarations_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'declaration_json'],
    num_rows: 10944
})

In [ ]:
import json

def get_person_metadata_from_dict_declaration(input_dict_declaration):
  name = input_dict_declaration['declaration']['general']['declarant']['nom']
  surname = input_dict_declaration['declaration']['general']['declarant']['prenom']
  if input_dict_declaration['declaration']['general']['qualiteDeclarantForPDF'] is None:
    return f"Fiche de {surname} {name} \n ------------ \n"
  else:
    title = input_dict_declaration['declaration']['general']['qualiteDeclarantForPDF']
    # debug json pretty print
    # print(json.dumps(input_dict_declaration, indent=2))

    return f"Fiche de {surname} {name} - {title} ".lower()

In [ ]:
# print(get_person_metadata_from_dict_declaration(parsed_json))

In [ ]:
def map_extract_text_from_json(row):
  parsed_json = json.loads(row['declaration_json'])
  text_index = get_person_metadata_from_dict_declaration(parsed_json)
  row['text_index'] = text_index
  return row

In [ ]:
declarations_ds = declarations_ds.map(map_extract_text_from_json, num_proc=2)

Map (num_proc=2):   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
if False:
  declarations_ds.push_to_hub("the-french-artist/hatvp_declarations_xml_plus_json_plus_index")

## check text extraction

In [ ]:
declarations_ds = load_dataset('the-french-artist/hatvp_declarations_xml_plus_json_plus_index', split='train')

Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
declarations_df = declarations_ds.to_pandas()

In [ ]:
declarations_df[['text_index']]

,text_index
0,damien abad - Député de l'Ain
1,damien abad - Député de l'Ain
2,caroline abadie - Députée de l'Isère
3,caroline abadie - Députée de l'Isère
4,joelle abadie - Elu départemental/Hautes-Pyré...
...,...
10939,nadia zourgui - Adjointe au maire de Strasbourg
10940,comlan hubert zoutu - Membre d’EPCI/Communaut...
10941,nicolas zuili - Adjoint au maire de Rouen
10942,nicolas zuili - Adjoint au maire de Rouen


## We find out that the job title is not good  

It turns out the the job title extracted from the XML files is an approximation.  
The actual job title, the one displayed online, comes from a secondary CSV file.  


In [ ]:
import pandas as pd

Below is the single use code used to upload the raw declaraction published list to HF HUB

In [ ]:
# base_csv = pd.read_csv('/content/liste (3).csv', sep=";")

In [ ]:
# from datasets import Dataset

# published_ds = Dataset.from_pandas(base_csv)
# published_ds

In [ ]:
# published_ds.push_to_hub("the-french-artist/hatvp_published_declarations")

Load published declarations dataset

In [ ]:
from datasets import load_dataset

published_ds = load_dataset('the-french-artist/hatvp_published_declarations', split='train')

Generating train split:   0%|          | 0/14818 [00:00<?, ? examples/s]

In [ ]:
published_df = published_ds.to_pandas()

We checked and it turns out many names and surnames are duplicates, which means that we will have a hard time getting the correct job title using these as our only index.  

Eg. 3 different François Dupont with different job titles.  

So we also use date_depot to get a better match.  

We also find that some people are missing a date_depot, so we filter them out and we'll see later.  

In [ ]:
published_df

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,date_depot,nom_fichier,url_dossier,open_data,statut_publication,id_origine,url_photo
0,Mme,Abbassia,HAKEM,hakemaaaabbassia4615,commune,Adjointe au maire de Nantes,di,44,2021-09-27,2020-09-01,hakem-abbassia-di16146-commune-nantes.pdf,/pages_nominatives/hakem-abbassia,hakem-abbassia-di16146-commune-nantes.xml,Livrée,None,None
1,Mme,Abbassia,HAKEM,hakemaaaabbassia4615,commune,Adjointe au maire de Nantes,dim,44,2021-09-28,2021-08-02,hakem-abbassia-dim16147-commune-nantes.pdf,/pages_nominatives/hakem-abbassia,hakem-abbassia-dim16147-commune-nantes.xml,Livrée,None,None
2,M.,Abdel Kader,CHEKHEMANI,chekhemaniaaaabdelkader1679,commune,Adjoint au maire de Rouen,di,76,2021-10-11,2020-10-14,chekhemani-abdel-kader-di16326-commune-rouen.pdf,/pages_nominatives/chekhemani-abdel-kader,chekhemani-abdel-kader-di16326-commune-rouen.xml,Livrée,None,None
3,M.,Abdel Kader,CHEKHEMANI,chekhemaniaaaabdelkader1679,commune,Adjoint au maire de Rouen,dim,76,2021-10-12,2021-07-14,chekhemani-abdel-kader-dim16327-commune-rouen.pdf,/pages_nominatives/chekhemani-abdel-kader,chekhemani-abdel-kader-dim16327-commune-rouen.xml,Livrée,None,None
4,M.,Abdelaziz,HAMIDA,HAMIDA Abdelaziz18939,commune,Maire de Goussainville,di,28,2021-08-10,2020-10-03,hamida-abdelaziz-di15206-commune-goussainville...,/pages_nominatives/hamida-abdelaziz-18939,hamida-abdelaziz-di15206-commune-goussainville...,Livrée,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14813,M.,Zartoshte,BAKHTIARI,BAKHTIARI Zartoshte18992,commune,Maire de Neuilly-sur-Marne,di,93,2022-11-15,2022-03-07,bakhtiari-zartoshte-di22174-commune-neuilly-su...,/pages_nominatives/bakhtiari-zartoshte-18992,bakhtiari-zartoshte-di22174-commune-neuilly-su...,Livrée,None,None
14814,M.,Zartoshte,BAKHTIARI,BAKHTIARI Zartoshte18992,commune,Maire de Neuilly-sur-Marne,dim,93,2022-11-16,2022-09-19,bakhtiari-zartoshte-dim22175-commune-neuilly-s...,/pages_nominatives/bakhtiari-zartoshte-18992,bakhtiari-zartoshte-dim22175-commune-neuilly-s...,Livrée,None,None
14815,Mme,Zemorda,KHELIFI,KHELIFI Zemorda18898,ctsp,Vice-Présidente de la Métropole de Lyon,di,69,2021-06-23,2020-09-19,khelifi-zemorda-di14099-ctsp-69.pdf,/pages_nominatives/khelifi-zemorda-18898,khelifi-zemorda-di14099-ctsp-69.xml,Livrée,None,None
14816,Mme,Zemorda,KHELIFI,KHELIFI Zemorda18898,ctsp,Vice-Présidente de la Métropole de Lyon,dim,69,None,None,None,/pages_nominatives/khelifi-zemorda-18898,None,En cours,None,None


In [ ]:
for_index_df = published_df[['prenom', 'nom', 'qualite', 'date_depot']]
for_index_df.columns = ['surname', 'name', 'title', 'upload_date']
for_index_df = for_index_df.drop_duplicates()
for_index_df = for_index_df[~for_index_df.upload_date.isna()]
for_index_df.head()

,surname,name,title,upload_date
0,Abbassia,HAKEM,Adjointe au maire de Nantes,2020-09-01
1,Abbassia,HAKEM,Adjointe au maire de Nantes,2021-08-02
2,Abdel Kader,CHEKHEMANI,Adjoint au maire de Rouen,2020-10-14
3,Abdel Kader,CHEKHEMANI,Adjoint au maire de Rouen,2021-07-14
4,Abdelaziz,HAMIDA,Maire de Goussainville,2020-10-03


In [ ]:
for_index_df['match_index'] = for_index_df['surname'].str.lower() + ' ' + for_index_df['name'].str.lower() + ' ' + for_index_df['upload_date']
for_index_df.head()

,surname,name,title,upload_date,match_index
0,Abbassia,HAKEM,Adjointe au maire de Nantes,2020-09-01,abbassia hakem 2020-09-01
1,Abbassia,HAKEM,Adjointe au maire de Nantes,2021-08-02,abbassia hakem 2021-08-02
2,Abdel Kader,CHEKHEMANI,Adjoint au maire de Rouen,2020-10-14,abdel kader chekhemani 2020-10-14
3,Abdel Kader,CHEKHEMANI,Adjoint au maire de Rouen,2021-07-14,abdel kader chekhemani 2021-07-14
4,Abdelaziz,HAMIDA,Maire de Goussainville,2020-10-03,abdelaziz hamida 2020-10-03


We then remove duplicate index values (due to possible differences in job titles)

In [ ]:
for_index_df_unique = for_index_df.drop_duplicates(subset='match_index')
for_index_df_unique

,surname,name,title,upload_date,match_index
0,Abbassia,HAKEM,Adjointe au maire de Nantes,2020-09-01,abbassia hakem 2020-09-01
1,Abbassia,HAKEM,Adjointe au maire de Nantes,2021-08-02,abbassia hakem 2021-08-02
2,Abdel Kader,CHEKHEMANI,Adjoint au maire de Rouen,2020-10-14,abdel kader chekhemani 2020-10-14
3,Abdel Kader,CHEKHEMANI,Adjoint au maire de Rouen,2021-07-14,abdel kader chekhemani 2021-07-14
4,Abdelaziz,HAMIDA,Maire de Goussainville,2020-10-03,abdelaziz hamida 2020-10-03
...,...,...,...,...,...
14811,Zaïnaba,SAÏD-ANZUM,Conseillère départementale de la Seine-Saint-D...,2023-09-10,zaïnaba saïd-anzum 2023-09-10
14812,Zamimou,AHAMADI,Vice-Présidente du conseil départemental de Ma...,2023-10-30,zamimou ahamadi 2023-10-30
14813,Zartoshte,BAKHTIARI,Maire de Neuilly-sur-Marne,2022-03-07,zartoshte bakhtiari 2022-03-07
14814,Zartoshte,BAKHTIARI,Maire de Neuilly-sur-Marne,2022-09-19,zartoshte bakhtiari 2022-09-19


We create a matching index on the declaration dataset:

In [ ]:
import json

def get_index(json_string):
  parsed_json = json.loads(json_string)
  name = parsed_json['declaration']['general']['declarant']['nom'].lower()
  surname = parsed_json['declaration']['general']['declarant']['prenom'].lower()
  upload_date = parsed_json['declaration']['dateDepot']
  upload_date_list = upload_date[:10].split('/')
  formatted_date = f"{upload_date_list[2]}-{upload_date_list[1]}-{upload_date_list[0]}"
  return surname + ' ' + name + ' ' + formatted_date


In [ ]:
declarations_df['match_index'] = declarations_df['declaration_json'].apply(get_index)

In [ ]:
declarations_df.head()

,xml_sha1,declaration_xml,declaration_json,text_index,match_index
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,"{""declaration"": {""dateDepot"": ""11/07/2022 15:4...",damien abad - Député de l'Ain,damien abad 2022-07-11
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,"{""declaration"": {""dateDepot"": ""27/11/2022 18:1...",damien abad - Député de l'Ain,damien abad 2022-11-27
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,"{""declaration"": {""dateDepot"": ""19/08/2022 10:0...",caroline abadie - Députée de l'Isère,caroline abadie 2022-08-19
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,"{""declaration"": {""dateDepot"": ""04/10/2022 17:2...",caroline abadie - Députée de l'Isère,caroline abadie 2022-10-04
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,"{""declaration"": {""dateDepot"": ""20/09/2021 13:4...",joelle abadie - Elu départemental/Hautes-Pyré...,joelle abadie 2021-09-20


We perform a left join on the matching index to get the correct job titles

In [ ]:
import pandas as pd

# Perform left join
declarations_with_title_df = declarations_df.merge(for_index_df_unique[['match_index', 'title']], on='match_index', how='left')

In [ ]:
assert len(declarations_with_title_df) == len(declarations_df)

In [ ]:
declarations_with_title_df.head()

,xml_sha1,declaration_xml,declaration_json,text_index,match_index,title
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,"{""declaration"": {""dateDepot"": ""11/07/2022 15:4...",damien abad - Député de l'Ain,damien abad 2022-07-11,Député de l'Ain
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,"{""declaration"": {""dateDepot"": ""27/11/2022 18:1...",damien abad - Député de l'Ain,damien abad 2022-11-27,Député de l'Ain
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,"{""declaration"": {""dateDepot"": ""19/08/2022 10:0...",caroline abadie - Députée de l'Isère,caroline abadie 2022-08-19,Députée de l'Isère
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,"{""declaration"": {""dateDepot"": ""04/10/2022 17:2...",caroline abadie - Députée de l'Isère,caroline abadie 2022-10-04,Députée de l'Isère
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,"{""declaration"": {""dateDepot"": ""20/09/2021 13:4...",joelle abadie - Elu départemental/Hautes-Pyré...,joelle abadie 2021-09-20,NaN


We now get the final text_index feature:  

- use the previous function to make the index, but use the title from feature "title" instead  
- if "title" is not present, we revert back to the XML source

In [ ]:
import numpy as np

def make_text_index(row):
  parsed_json = json.loads(row['declaration_json'])
  name = parsed_json['declaration']['general']['declarant']['nom'].lower()
  surname = parsed_json['declaration']['general']['declarant']['prenom'].lower()
  if row['title'] is not np.nan:
    title = row['title']
  elif parsed_json['declaration']['general']['qualiteDeclarantForPDF'] is not None:
    title = parsed_json['declaration']['general']['qualiteDeclarantForPDF']
  else:
    title = ''
  return surname + ' ' + name + ' - ' + ' ' + title.lower()

In [ ]:
declarations_with_title_df['text_index'] = declarations_with_title_df.apply(lambda row: make_text_index(row), axis=1)

In [ ]:
del declarations_with_title_df['match_index']
del declarations_with_title_df['title']

In [ ]:
declarations_with_title_df.head()

,xml_sha1,declaration_xml,declaration_json,text_index
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,"{""declaration"": {""dateDepot"": ""11/07/2022 15:4...",damien abad - député de l'ain
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,"{""declaration"": {""dateDepot"": ""27/11/2022 18:1...",damien abad - député de l'ain
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,"{""declaration"": {""dateDepot"": ""19/08/2022 10:0...",caroline abadie - députée de l'isère
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,"{""declaration"": {""dateDepot"": ""04/10/2022 17:2...",caroline abadie - députée de l'isère
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,"{""declaration"": {""dateDepot"": ""20/09/2021 13:4...",joelle abadie - elu départemental/hautes-pyré...


In [ ]:
from datasets import Dataset
declarations_with_title_ds = Dataset.from_pandas(declarations_with_title_df)

In [ ]:
declarations_with_title_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'declaration_json', 'text_index'],
    num_rows: 10944
})

In [ ]:
if True: #change to true to overwrite dataset
  declarations_with_title_ds.push_to_hub("the-french-artist/hatvp_declarations_xml_plus_json_plus_index")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/456 [00:00<?, ?B/s]